# FFN Analysis
- <b>Date:</b> 10/24/2022
- <b><u>Table of Contents</u></b>
    1. [Planning](#sec1)
    1. [Title1](#sec2)
    1. [Title1](#sec3)
    1. [Title1](#sec4)
    1. [Title1](#sec5)

### Goal #1: Write a function that takes a reading list
- separates AO3 link and everything else
- checks for duplicates in existing storages & de-dups
- separates explicit and non-explicit 
- grabs all data for each fic & writes it to a csv of 'unsorted'

### Goal #2: Write sorting function 
- that displays one fic from either explicit or non-explicit
- take user input, add dtb tag & reread boolean & important
    - ffn, to read, author, tag, coffee, cont read, unsure
    - reread?
    - star
- 


- ao3 id
- fandoms []


### Goals (current)
    1. [Clean-Incoming-Data functions] organizer & create fic csv/json forms 
        
    1. [Display & Interface functions] sort reading list URLs, get masterlist of works, authors, tags, others?
        - [ ] def getInput(allowedList)  <--- current
        - [ ] def read_dtb(dtbName)
        - [ ] def display_fic(ficId)
        - [ ] def start_sort_temp()
        - nice interface, main caller

- maybe sort out/differently store series
    1. [Search functions] function to go through unsorted lists to sort them into dtbs (NOT in-depth)
        - [ ] def saveSort(newInfo)
        - [ ] def edit_fic(ficId)
        - have an edit method as well
        - have a history?
        - have a report about all lists
    1. [Cleaner & Testing functions]
        - def check_all_dtb()
            - for cleaning (?), check for duplicates
            - def dtb_check_dups(dtbName)
        - def dtb_report()
            - prints out/writes (?) number of fics, names of recently added? 
    1. [Fic functions]
        - def fic_identify(type, info)
            - type: title, id, url
            - returns fic_id
        - def fic_report(fic_id)
            - print report: current dtb, date moved, fic info (title, author, fandom, etc.)
        - def interate_author(author_name)
            - gets all works not already in dtb of author, displays each other, user chooses if/where they want to add it

### Imports, Variables, & Helper Functions

In [ ]:
%run helpers.ipynb

In [ ]:
# VARIABLES
key_shorts = {"f": "ffn",
            "tr": "to_read",
            "cr": "continue_read",  
            "c": "coffee",
            "d": "unsure"
             }
# author, favorite_tag, explore_tag

### General Workspace

## CURRENT GOAL: Make pd DF/csv for all MASTERS
- Make other one a single-use & store it

In [ ]:
import json
from datetime import datetime

def add_to_masterfiles(urlFile):
    """
    Takes ONE txt file of URLs, appends new urls (probably from a new reading list) to the 3 MASTER json files:
    MASTER_noDupURLs, MASTER_noDupWorks, MASTER_others. Pair with `combineToTxt(dirPath)` to convert whole 
    folders of TXT files.
    Returns 'success' if successful. 
    """
    # Get current date & initialize 3 lists
    now = datetime.now()
    date_str = f"<Added: {now.strftime('%m-%d-%y %H:%M:%S')}>"
    
    
    # Initialize variables
    files = ["MASTER_noDupURLs.json", "MASTER_noDupWorks.json", "MASTER_others.json"]
    
    for file in files:
        if not os.path.isfile(file):
            with open(file,"w") as outfile:
                json.dump([], outfile)
            print(f"Made {file}")
    
    newNoDupURLs = []
    newNoDupWorks = []
    newOthers = []

    
    # Read in original files
    with open(files[0], "r") as infile:
        noDupURLs = json.load(infile)
    
    # rules a little different for noDupWorks bc it's formatted: [[typeI, url], ...
    with open(files[1], "r") as infile:  
        noDupWorks = json.load(infile)
        if noDupWorks == []: typeIdList = []
        else: 
            typeIdList = list(list(zip(*noDupWorks))[0])
        
    with open(files[2], "r") as infile:
        others = json.load(infile)

    totalLen = 0
    # Read in new URLs 
    with open(urlFile, "r") as infile:
        for line in infile:
            line = line.strip() #to get rid of \n
#             print(line) #DID SOMETHING GO WRONG?
            # if not an AO3 url
            if "archiveofourown.org" not in line:
                if line not in others:
                    newOthers.append(line)

            else:
                # should add to no-duplicate-url file?
                if line not in noDupURLs:
                    newNoDupURLs.append(line)

                # should add to no-duplicate-work file?
                typeId = list(getTypeAndId(line))
                if typeId not in typeIdList:
                    typeIdList.append(typeId)
                    pair = [typeId, url]
                    newNoDupWorks.append(pair)
            totalLen += 1
    
    # Format & Write newly added-to files
    fileTypes = [[noDupURLs, newNoDupURLs, files[0]], 
                 [noDupWorks, newNoDupWorks, files[1]], 
                 [others, newOthers, files[2]]]
        
    for original, new, file in fileTypes:
        original.append(date_str) # add date stamp
        original.extend(new) # add new URLs
        
        # Write newly appended-lists
        with open(file, "w") as infile:
            json.dump(original, infile)
        
    # print addition report
    print(f"There were {totalLen} url(s) in '{urlFile}'")
    print(f"Added {len(newNoDupURLs)} url(s) to MASTER_noDupURLs.json")
    print(f"Added {len(newNoDupWorks)} url(s) to MASTER_noDupWorks.json")
    print(f"Added {len(newOthers)} url(s) to MASTER_others.json")
    
    return "success"

    
# add_to_masterfiles("txtOutput_12-25-22.txt")

## Prelim-sorting Interface workspace

In [ ]:
def getInput():
    """ Gets all the user answers for the v9.1 prelim sorting interface
    Takes nothing. Returns nothing."""
    passes = False
    
    # get dtb input
    print("Database: ")
    getCorrectInput(list(key_shorts.keys()))
    
    # get reread input
    print("Reread?: ")
    getCorrectInput(["r",""])
    
    # get star input
    print("Star?: ")
    getCorrectInput(["s",""])
    

In [ ]:
def displayFic():
    # display stuff
    print("here")
    # get input
    dtb = "-1"
    reread = "-1"
    star = "-1"
    
    passes = 0
    while passes == 0:
        passes = 1
        ans = getInput()
        dtb = ans[0].replace(" ","").split()
        for dtb_tag in dtb:
            if dtb_tag not in key_shorts.keys():
                print(f" - '{dtb_tag}' not in key shortcuts")
                passes = 0
        if ans[1] != "r" or ans[1] != None:
            print(f" - it's 'r' or nothing! (for reread)")
        if ans[2] != "s" or ans[2] != "":
            print(f" - it's 's' or nothing! (for star)")
        if passes == 0:
            print(f" - shortcuts: {key_shortcuts}")
        
        

## AO3 Workspace

In [ ]:
import AO3
url = "https://archiveofourown.org/works/13273611/chapters/30371190"
workid = AO3.utils.workid_from_url(url)
work = AO3.Work(workid)

attrs = dir(work)[33:]

In [ ]:
import pandas as pd
import json
import datetime

with open("newRemain.json") as infile:
    newRemain = json.load(infile)


temp = []
with open(masterNoDupUrls) as infile:
    data = json.load(infile)
    for line in data:
        if "<Added:" not in line:
            temp.append(line)

def ten(x):
    if x in newRemain: return datetime.datetime(2022, 1, 11)
    else: return datetime.datetime(2017, 5, 31)
        
df1 = pd.DataFrame(temp)
df1 = df1.rename(columns={0: "url"})
df1["date_added"] = df1["url"].apply(ten)
df1

In [ ]:
def yu(x):
    return getTypeAndId(x)[1]

df1["id"] = df1["url"].apply(yu)
df1

In [ ]:
def xu(x):
    return getTypeAndId(x)[0]

df1["type"] = df1["url"].apply(xu)
df1["type"].unique()

In [ ]:
def typeToWorkType(x):
    if x == "users": return "user"
    elif x == "collections": return "collection"
    elif x == "search": return "search"
    elif x == "tags": return "tag"
    elif x == "series": return "series"
    elif x == "works": return "work"
    elif x == "chapters": return "work"
    elif x == "external_works": return "work"
    elif 'collections:' in x: return "work"
    elif x == "comments": return "comments"
    else: return "UNDEFINED"
    
df1["work_type"] = df1["type"].apply(zu)

In [ ]:
df1.rename(columns={"type": "url_type"}, inplace=True)

In [ ]:
df1[df1["id"] == "thepartyresponsible"]["url"].to_list()

In [ ]:
#df1.to_csv("MASTER_noDupURLs.csv")

collection, work, series, user, tags, search, col_work, external_work, comments, chapters

add pseuds

<u>url_type</u>
1. work
1. chapters
1. colWorks
1. external work
1. user
1. collection
1. search
1. tag
1. series

<u>work_type</u>
1. work (regular, chapters, colWorks, external_work)
1. user
1. collection
1. series
1. OTHER: search, tag, comments

cols = url, work_type, url_type, id

In [ ]:
df1

In [ ]:
import AO3

url = "https://archiveofourown.org/works/14392692/chapters/33236241"
workid = AO3.utils.workid_from_url(url)
print(f"Work ID: {workid}")
work = AO3.Work(workid)
print(f"Chapters: {work.title}")




In [ ]:
# load in newer urls
with open("newRemain.json") as infile:
    newRemain = json.load(infile)

# load in all urls
temp = []
with open("jsonOutput_12-27-22.json") as infile:
    data = json.load(infile)
    for line in data:
        if "<Added:" not in line:
            temp.append(line)

# define func to assign dates
def ten(x):
    if x in newRemain: return datetime(2022, 1, 11)
    else: return datetime(2017, 5, 31)

# load URLs into DF
df1 = pd.DataFrame(temp)
df1 = df1.rename(columns={0: "url"})

# add date_added
df1["date_added"] = df1["url"].apply(ten)
df1

In [ ]:
df1.shape

In [ ]:
# get other database
now = datetime.now()
fileName = f"others_{now.strftime('%m-%d-%y %H:%M:%S')}.csv"

dfOther = df1.loc[df1["url"].str.contains("archiveofourown.org") == False]
dfOther.reset_index().drop(columns=["index"])

dfOther.to_csv(fileName)

In [ ]:
df1 = df1.loc[df1["url"].str.contains("archiveofourown.org")]
df1 = df1.reset_index().drop(columns=["index"])
df1

In [ ]:
%run helpers.ipynb

In [ ]:
# add work_type, url_type, id
def yu(x):
    return getTypeAndId(x)[0]

df1["url_type"] = df1["url"].apply(yu)
df1

In [ ]:
def typeToWorkType(x):
    if x == "users": return "user"
    elif x == "collections": return "collection"
    elif x == "search": return "search"
    elif x == "tags": return "tag"
    elif x == "series": return "series"
    elif x == "works": return "work"
    elif x == "chapters": return "work"
    elif x == "external_works": return "work"
    elif 'collections:' in x: return "work"
    elif x == "comments": return "comments"
    else: return "UNDEFINED"
    
df1["work_type"] = df1["url_type"].apply(typeToWorkType)

In [ ]:
df1.to_csv("all-urls.csv")

* first_df1.csv - holds all non-duplicates AO3 urls as of 12/29/2022

- **url:** url of fic
- **date_added:** date url was imported from readinglist
- **id:** id of work
- **url_type:** 
- **work_type:** 


In [ ]:
import pandas as pd
df1 = pd.read_csv("first_df1.csv").drop_duplicates("url").reset_index().drop(columns=["Unnamed: 0","index"])
df1

In [ ]:
# make noDupUrls
dfUrls = df1
dfUrls = dfUrls.drop_duplicates(["url"])


In [ ]:
df1.sort_values(by=['url_type',"id"], ascending=False).to_excel("tester.xlsx")

In [ ]:
test = tets.merge(dfUrls, how='outer')

In [ ]:
# make noDupWorks
dfWorks = df1
dfWorks.drop_duplicates(subset=[""]).shape